# Getting data in to a graph

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    import tensorflow as tf

graph = tf.get_default_graph()
session = tf.Session()

> Analogy: The graph is like a blueprint and a session is like a construction site

In [ ]:
graph.get_operations()

We haven not added any operations to the graph yet

In [ ]:
zero_d = tf.constant(4)

`tf.constant` adds an operation to the graph which creates a constant of 4

In [ ]:
graph.get_operations()

In [ ]:
session.run(zero_d - 1)

After running something in the session, do we expect more operations in the graph?

In [ ]:
graph.get_operations()

In [ ]:
session.run(zero_d, feed_dict={zero_d: 9})

How about this? Will this have added an operation to the graph?

In [ ]:
graph.get_operations()

Disclaimer: This example gleefully uses `tf.constant()` and not `tf.placeholder()`. Really if you plan to feed data in, you should use `tf.placeholder()`. Also you likely want to provide more useful `name` values for things, etc.

Here we are using a plain python list. The type on the left is a TensorFlow tensor.

In [ ]:
one_d = tf.constant([1, 1])

`one_d + [2, 2]` wouldn't be a valid op in standard python. The list is sent to the graph, cast so the addition operation is valid i.e. is done with tensors.

In [ ]:
session.run(one_d + [2, 2])

In [ ]:
import numpy as np

two_d = tf.constant(np.array([[2, 2], [2, 2]]))

In [ ]:
session.run(two_d + [[1, 1], [1, 1]])

In [ ]:
session.run(two_d, feed_dict={two_d: [[9, 9], [9, 9]]})

---

# Getting data in using TFRecords

In [ ]:
# Python dict:
my_dict = {'features' : {
    'my_ints': [5, 6],
    'my_float': [2.7],
    'my_bytes': [b'data']
}}

In [ ]:
# TFRecords `Example`:
my_example = tf.train.Example(features=tf.train.Features(feature={
    'my_ints': tf.train.Feature(int64_list=tf.train.Int64List(value=[5, 6])),
    'my_float': tf.train.Feature(float_list=tf.train.FloatList(value=[2.7])),
    'my_bytes': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'data']))
}))

In [ ]:
image = np.ones((1, 28, 28), dtype=np.uint8)  # not a meaningful image
label = 9  # for a classification problem

image_bytes = image.tostring()
image_shape = image.shape

my_example = tf.train.Example(features=tf.train.Features(feature={
    'image_bytes': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes])),
    'image_shape': tf.train.Feature(int64_list=tf.train.Int64List(value=image_shape)),
    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
}))

In [ ]:
my_example_str = my_example.SerializeToString()
with tf.python_io.TFRecordWriter('an_example.tfrecords') as writer:
    writer.write(my_example_str)

reader = tf.python_io.tf_record_iterator('an_example.tfrecords')
those_examples = [tf.train.Example().FromString(example_str)
                  for example_str in reader]

In [ ]:
same_example = those_examples[0]
same_image_bytes = same_example.features.feature['image_bytes'].bytes_list.value[0]
same_image_shape = list(same_example.features.feature['image_shape'].int64_list.value)
same_label = same_example.features.feature['label'].int64_list.value[0]
same_image = np.frombuffer (same_image_bytes, dtype=np.uint8)
same_image.shape = same_image_shape

In [ ]:
same_label

In [ ]:
import os
os.remove('an_example.tfrecords')

In [ ]:
help(tf.TFRecordReader)

 * Search: https://www.tensorflow.org/s/results/?q=TFRecordReader
 * Docs: https://www.tensorflow.org/api_docs/python/tf/TFRecordReader
 * Source: https://github.com/tensorflow/tensorflow/blob/r1.4/tensorflow/python/ops/io_ops.py#L413

__Exercies__

Can you figure out what's in mytery.tfrecords?

In [ ]:
# just to get you started
reader = tf.python_io.tf_record_iterator('../assets/mystery.tfrecords')